In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from sepal_ui import sepalwidgets as sw
from sepal_ui.frontend.resize_trigger import ResizeTrigger

In [ ]:
#from sepal_ui import sepalwidgets as sw
from component.message import cm
#Import models
from component.model.aoi_date_model import AoiDateModel
from component.model.aux_model import AuxModel
from component.model.alerts_model import AlertFilterModel, SelectedAlertsModel
from component.model.analysis_model import AlertAnalysisModel
from component.model.app_model import AppTileModel


#Import tiles
from component.tile.aoi_tile import AoiTile
from component.tile.aux_tile import AuxTile
from component.tile.recipe_tile import RecipeTile
from component.tile.alerts_filter_tile import AlertsFilterTile
from component.tile.overview_tile import OverviewTile
from component.tile.analysis_tile import AnalysisTile

#Import custom App
from component.widget.custom_sw import CustomApp, CustomDrawerItem, CustomAppBar


# link tiles and models
from traitlets import Any, HasTraits, Unicode, link, observe

# Create an appBar
app_bar = CustomAppBar(cm.app.title, cm)

#Create models
aux_model = AuxModel()
aoi_date_model = AoiDateModel()
alert_filter_model = AlertFilterModel()
selected_alerts_model = SelectedAlertsModel()
analyzed_alerts_model = AlertAnalysisModel()
app_tile_model = AppTileModel()

#Create tiles
aux_tile = AuxTile(aux_model)
aoi_tile = AoiTile(aoi_date_model, alert_filter_model, aux_model, app_tile_model)
alert_filter_tile = AlertsFilterTile(aoi_date_model, aux_model, alert_filter_model, selected_alerts_model, analyzed_alerts_model, app_tile_model)
analysis_tile = AnalysisTile(aux_model, selected_alerts_model, analyzed_alerts_model, app_tile_model)
overview_tile = OverviewTile(aoi_date_model, analyzed_alerts_model, selected_alerts_model, aux_model, app_tile_model)
recipe_tile = RecipeTile(aux_model , aoi_date_model, alert_filter_model, selected_alerts_model, analyzed_alerts_model, app_tile_model, aux_tile, aoi_tile, alert_filter_tile, analysis_tile, overview_tile)
about_tile = sw.TileAbout(f"utils/{cm.about.pathname}")
disclaimer_tile = sw.TileDisclaimer()


#Link maps
link((aoi_tile.map_1, "center"), (alert_filter_tile.map_1, "center"))
link((aoi_tile.map_1, "zoom"), (alert_filter_tile.map_1, "zoom"))
link((aoi_tile.map_1, "center"), (overview_tile.map_1, "center"))
link((aoi_tile.map_1, "zoom"), (overview_tile.map_1, "zoom"))

# Gather all the partial tiles that you created previously
app_content = [about_tile, disclaimer_tile, recipe_tile, aux_tile, aoi_tile, alert_filter_tile, analysis_tile, overview_tile]
# create a drawer for each group of tile
# fmt: off
items = [
    CustomDrawerItem(app_tile_model, title = cm.app.drawer_item.about, icon = "fa-solid fa-question-circle", card="about_tile"),
    CustomDrawerItem(app_tile_model, title = cm.app.drawer_item.recipes, icon = "fa-solid fa-folder-open", card="recipe_tile"),
    CustomDrawerItem(app_tile_model, title = cm.app.drawer_item.aux, icon = "fa-solid fa-gear", card="aux_tile"),
    CustomDrawerItem(app_tile_model, title = cm.app.drawer_item.aoi, icon = "fa-solid fa-globe", card="aoi_tile"),
    CustomDrawerItem(app_tile_model, title = cm.app.drawer_item.filter, icon = "fa-solid fa-filter", card="filter_alerts"),
    CustomDrawerItem(app_tile_model, title = cm.app.drawer_item.overview, icon = "fa-solid fa-map-location-dot", card="overview_tile"),
    CustomDrawerItem(app_tile_model, title = cm.app.drawer_item.analysis, icon = "fa-solid fa-pen", card="analysis_tile"),
]
# fmt: on

# Add the links to the code, wiki and issue tracker of your
code_link = "https://github.com/SerafiniJose/defoAlerts/"
wiki_link = "https://github.com/SerafiniJose/defoAlerts/blob/master/doc/en.rst"
issue_link = "https://github.com/SerafiniJose/defoAlerts/issues/new"

# Create the side drawer with all its components
app_drawer = sw.NavDrawer(items=items, code=code_link, wiki=wiki_link, issue=issue_link)
# build the Html final app by gathering everything
app = CustomApp(
    app_tile_model,
    tiles=app_content,
    appBar=app_bar,
    navDrawer=app_drawer,
    translator=cm,
)
# display the app
app.show_tile("about_tile")#.add_banner(cm.app.banner)